# Lab 05: Local Search (Beam Search, Hill Climbing, Genetic Algorithm) Solutions


In [6]:
import random
import math

########## Task 01: Beam Search for Chess ##########

def evaluate_board(board):
    # Dummy board evaluation: returns a random score
    return random.uniform(-1, 1)

def generate_moves(board):
    # Dummy move generation: returns a list of dummy moves (e.g., strings)
    return [f"move_{i}" for i in range(1, 6)]

def beam_search_chess(initial_board, beam_width, depth_limit):
    beam = [(initial_board, [], 0)]  # Each element is (current_board, move_sequence, evaluation_score)
    
    for depth in range(depth_limit):
        new_beam = []
        for board, seq, score in beam:
            moves = generate_moves(board)
            for move in moves:
                # In a true implementation, apply move to board
                new_board = board + " | " + move
                new_score = evaluate_board(new_board)
                new_seq = seq + [move]
                new_beam.append((new_board, new_seq, new_score))
        # Keep only top beam_width candidates
        beam = sorted(new_beam, key=lambda x: x[2], reverse=True)[:beam_width]
        
    best_board, best_seq, best_score = beam[0]
    return best_seq, best_score

# Example usage for beam search
initial_board = "Initial Chess Board State"
beam_width = 3
depth_limit = 3
best_moves, score = beam_search_chess(initial_board, beam_width, depth_limit)

print("--- Beam Search for Chess ---")
print(f"Best move sequence: {best_moves}")
print(f"Evaluation score: {score}\n")

--- Beam Search for Chess ---
Best move sequence: ['move_5', 'move_2', 'move_4']
Evaluation score: 0.9930062000002606



In [11]:
########## Task 02: Hill Climbing for Delivery Route ##########

def total_distance(route):
    distance = 0
    for i in range(len(route) - 1):
        x1, y1 = route[i]
        x2, y2 = route[i+1]
        # Euclidean distance
        distance += math.hypot(x2 - x1, y2 - y1)
    return distance

def generate_neighbors(route):
    # Swap two cities to generate a neighbor
    neighbors = []
    n = len(route)
    for i in range(n):
        for j in range(i+1, n):
            new_route = route.copy()
            new_route[i], new_route[j] = new_route[j], new_route[i]
            neighbors.append(new_route)
    return neighbors

def hill_climbing_route(delivery_points):
    current_route = delivery_points.copy()
    current_distance = total_distance(current_route)
    improved = True
    iteration = 0
    
    while improved:
        improved = False
        best_neighbor = current_route
        for neighbor in generate_neighbors(current_route):
            d = total_distance(neighbor)
            if d < current_distance:
                current_distance = d
                best_neighbor = neighbor
                improved = True
        current_route = best_neighbor
        iteration += 1
        # To avoid infinite loops in case of plateaus
        if iteration > 1000:
            break
    return current_route, current_distance

# Example usage for hill climbing
# A list of (x, y) coordinates for delivery points
delivery_points = [(0,0), (1,3), (4,3), (6,1), (3,0)]
best_route, distance = hill_climbing_route(delivery_points)

print("--- Hill Climbing for Delivery Route ---")
print(f"Optimized route: {best_route}")
print(f"Total distance: {distance}\n")

--- Hill Climbing for Delivery Route ---
Optimized route: [(0, 0), (1, 3), (4, 3), (6, 1), (3, 0)]
Total distance: 12.15298244508295



In [12]:
########## Task 03: Genetic Algorithm for TSP ##########
import random
def create_individual(city_list):
    individual = city_list.copy()
    random.shuffle(individual)
    return individual

def calculate_fitness(individual):
    # Lower fitness is better (total route distance)
    distance = total_distance(individual + [individual[0]])  # return to start
    return distance

def crossover(parent1, parent2):
    # Ordered crossover
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]
    fill_pos = end
    for city in parent2:
        if city not in child:
            if fill_pos >= size:
                fill_pos = 0
            child[fill_pos] = city
            fill_pos += 1
    return child

def mutate(individual, mutation_rate=0.1):
    individual = individual.copy()
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(individual)-1)
            individual[i], individual[j] = individual[j], individual[i]
    return individual

def genetic_algorithm_tsp(city_list, population_size=50, generations=500, mutation_rate=0.1):
    # Create initial population
    population = [create_individual(city_list) for _ in range(population_size)]
    
    for generation in range(generations):
        # Evaluate fitness for each individual
        fitness_scores = [calculate_fitness(ind) for ind in population]
        
        # Selection: choose parents via tournament selection
        selected = []
        for _ in range(population_size):
            i, j = random.sample(range(population_size), 2)
            selected.append(population[i] if fitness_scores[i] < fitness_scores[j] else population[j])
        
        # Generate new population
        new_population = []
        for _ in range(0, population_size, 2):
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
        population = new_population
    
    # Get the best solution
    best_individual = min(population, key=calculate_fitness)
    best_distance = calculate_fitness(best_individual)
    return best_individual, best_distance

# Example usage for TSP Genetic Algorithm
# Generate 10 cities with random coordinates
city_list = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(10)]
best_route, best_distance = genetic_algorithm_tsp(city_list)

print("--- Genetic Algorithm for TSP ---")
print(f"Cities: {city_list}")
print(f"Best route: {best_route}")
print(f"Total distance: {best_distance}")

--- Genetic Algorithm for TSP ---
Cities: [(35, 26), (24, 43), (46, 49), (78, 3), (77, 88), (16, 67), (12, 10), (72, 92), (6, 69), (47, 48)]
Best route: [(6, 69), (35, 26), (78, 3), (47, 48), (77, 88), (72, 92), (46, 49), (12, 10), (24, 43), (16, 67)]
Total distance: 384.2768513145981
